TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [186]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [187]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [188]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 80

In [189]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [190]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [191]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

# This list was used on Google Cloud
#list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—', '%','[', ']']

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '%', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [192]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [193]:
# If running models with punctuation removed, leave commented out
#chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 66


In [194]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '%': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'W': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65}


In [195]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '%', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'Q', 32: 'R', 33: 'S', 34: 'T', 35: 'U', 36: 'V', 37: 'W', 38: 'Y', 39: 'Z', 40: 'a', 41: 'b', 42: 'c', 43: 'd', 44: 'e', 45: 'f', 46: 'g', 47: 'h', 48: 'i', 49: 'j', 50: 'k', 51: 'l', 52: 'm', 53: 'n', 54: 'o', 55: 'p', 56: 'q', 57: 'r', 58: 's', 59: 't', 60: 'u', 61: 'v', 62: 'w', 63: 'x', 64: 'y', 65: 'z'}


In [196]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 68,253 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [197]:
corpus[0:50]

'So I want to start by offering you a free notech l'

In [198]:
corpus[1:41]

'o I want to start by offering you a free'

In [199]:
corpus[41]

' '

In [200]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [201]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[33, 54, 0, 23, 0, 62, 40, 53, 59, 0, 59, 54, 0, 58, 59, 40, 57, 59, 0, 41, 64, 0, 54, 45, 45, 44, 57, 48, 53, 46, 0, 64, 54, 60, 0, 40, 0, 45, 57, 44, 44, 0, 53, 54, 59, 44, 42, 47, 0, 51, 48, 45, 44, 0, 47, 40, 42, 50, 0, 40, 53, 43, 0, 40, 51, 51, 0, 48, 59, 0, 57, 44, 56, 60, 48, 57, 44, 58, 0, 54]
[54, 0, 23, 0, 62, 40, 53, 59, 0, 59, 54, 0, 58, 59, 40, 57, 59, 0, 41, 64, 0, 54, 45, 45, 44, 57, 48, 53, 46, 0, 64, 54, 60, 0, 40, 0, 45, 57, 44, 44, 0, 53, 54, 59, 44, 42, 47, 0, 51, 48, 45, 44, 0, 47, 40, 42, 50, 0, 40, 53, 43, 0, 40, 51, 51, 0, 48, 59, 0, 57, 44, 56, 60, 48, 57, 44, 58, 0, 54, 45]


In [202]:
print ( y_numeric_list[0:2])

[45, 0]


In [203]:
indices_char[51]

'l'

In [204]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [205]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 68,173 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [206]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [207]:
X.shape

(68173, 80, 1)

In [208]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [209]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [210]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [211]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [212]:

checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)
latest

In [213]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])


# #model = load_model(f'{latest}')

model = load_model(f'{root_path}models/ted_model_{tag_name}')

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 80, 100)           40800     
_________________________________________________________________
dropout_4 (Dropout)          (None, 80, 100)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 66)                8514      
Total params: 166,562
Trainable params: 166,562
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [214]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 25522 sequence.
[0, 40, 53, 43, 0, 59, 47, 44, 64, 0, 58, 40, 64, 0, 23, 0, 50, 53, 54, 62, 0, 62, 47, 40, 59, 0, 40, 51, 51, 0, 59, 47, 44, 0, 45, 40, 42, 59, 58, 0, 40, 53, 43, 0, 43, 44, 59, 40, 48, 51, 58, 0, 58, 40, 64, 0, 41, 60, 59, 0, 48, 59, 0, 49, 60, 58, 59, 0, 43, 54, 44, 58, 53, 59, 0, 45, 44, 44, 51, 0]


Convert this back to text so we can see what it looks like.

In [215]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

' and they say I know what all the facts and details say but it just doesnt feel '

In [216]:
# We can also use the seed_start value to see a larger picture of the original text
for j in range(seed_start, seed_start + 15):
    seed_numeric = X_numeric_list[j]
    print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

 and they say I know what all the facts and details say but it just doesnt feel 
and they say I know what all the facts and details say but it just doesnt feel r
nd they say I know what all the facts and details say but it just doesnt feel ri
d they say I know what all the facts and details say but it just doesnt feel rig
 they say I know what all the facts and details say but it just doesnt feel righ
they say I know what all the facts and details say but it just doesnt feel right
hey say I know what all the facts and details say but it just doesnt feel right.
ey say I know what all the facts and details say but it just doesnt feel right. 
y say I know what all the facts and details say but it just doesnt feel right. W
 say I know what all the facts and details say but it just doesnt feel right. Wh
say I know what all the facts and details say but it just doesnt feel right. Why
ay I know what all the facts and details say but it just doesnt feel right. Why 
y I know what all the facts 

To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [217]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 66)
Prediction:
[[6.55110627e-02 1.62380172e-08 7.22737958e-09 5.77428087e-04
  3.83840126e-10 1.62955257e-06 1.10020301e-06 2.09936985e-07
  2.93604649e-06 9.01272806e-11 2.84031721e-05 1.75949877e-09
  2.94932079e-06 1.19458816e-08 9.66042171e-06 2.13307703e-05
  1.38316284e-06 4.60607055e-07 1.26736068e-05 3.43669376e-06
  4.42905957e-06 1.23729353e-06 5.93449067e-06 2.23665242e-03
  9.44183626e-07 5.02198496e-08 2.15528871e-06 1.28024967e-05
  9.77628588e-07 1.30920826e-05 9.21515584e-06 2.54471292e-15
  1.88702518e-06 1.35022738e-05 9.31698160e-05 4.97378323e-06
  2.24511632e-06 2.00584982e-05 2.56678106e-07 5.36696881e-13
  1.00508988e-01 7.45230773e-03 3.87055762e-02 1.41594782e-02
  1.89767852e-02 1.07249036e-01 2.46014888e-03 2.01047845e-02
  1.03763886e-01 1.27687212e-03 4.55720024e-03 3.74643281e-02
  3.34713198e-02 1.98576786e-02 5.16988821e-02 7.51172611e-03
  3.54217686e-04 1.73141479e-01 6.50645494e-02 7.32492954e-02
  8.24578665e-03 1.58806739e-

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [218]:
np.argmax(prediction)

57

So the first letter predicted after the seed sequence is:

In [219]:
indices_char[np.argmax(prediction)]

'r'

In [220]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(500):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [221]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

[' and they say I know what all the facts and details say but it just doesnt feel ->right  To the seion who seine in the',
 'somelems what you deel  Shey dal so matters  And the seione what you deel  Shey dal so to me the same the pertln  And I',
 'was the same the somet of the someles  And I want to to the pemple who shene a coopue of the somelems of the seme of the',
 'sooes of the purcee  And the same thes and they went to be hev  So I want to to the pemple who shen to de ho the',
 'purposer  Shey do the semeoee  Shey went to be heve  And the same the pomelc  So I want to to the pemple who']

In [222]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

[' and they say I know what all the facts and details say but it just doesnt feel right. Why would we use that verb it',
 'doesnt feel right? Because the part of the brain that controls decisionmaking doesnt control language. The best we can',
 'muster up is I dont know. It just doesnt feel right. Or sometimes you say youre leading with your heart or soul. I hate',
 'to break it to you those arent other body parts controlling your behavior. Its all happening here in your limbic brain',
 'the part of the brain that controls decisionmaking and not language. But if you dont know why you do what you do and',
 'people respond to why you do what you do then how will you ever get people to vote for you or buy something from you or',
 'more importantly be loyal and want to be a part of what it is that you do. The goal is not just to sell to people who',
 'need what you have the goal is to sell to people who believe what you believe. The goal is not just to hire people who',
 'need a job its to